In [1]:
:load  ParseLoc

### create Location for test

In [2]:
x = Location {database = "ushvr12", dbdesc = "dev", host = "odp-us-innovation-postgres-db.cxzrm3enxxmy.us-east-1.rds.amazonaws.com", loc_name = "sedhp", loc_class = "oracle", loc_directory = "/u01/app/oracle/product/19.3.0.0/dbee_1", loc_remote_node = "ora-mke2-scanp.am.health.ge.com", loc_remote_login = "gehc_hvr5", loc_remote_pwd = "", loc_remote_port = 4344, loc_db_name = "", loc_db_user = "PLANT_USER_ODP/!{r/2axxMra/8grrk/}!@ora-mke2-scanp.am.health.ge.com:1521/prd1mes", loc_description = "scdhp connection"}

####  Visualize Location

In [3]:
import Data.Text.Prettyprint.Doc

pretty x

Location {
  database = ushvr12
  dbdesc = dev
  host = odp-us-innovation-postgres-db.cxzrm3enxxmy.us-east-1.rds.amazonaws.com
  loc_name = sedhp
  loc_class = oracle
  loc_directory = /u01/app/oracle/product/19.3.0.0/dbee_1
  loc_remote_node = ora-mke2-scanp.am.health.ge.com
  loc_remote_login = gehc_hvr5
  loc_remote_pwd = 
  loc_remote_port = 4344
  loc_db_name = 
  loc_db_user = PLANT_USER_ODP/!{r/2axxMra/8grrk/}!@ora-mke2-scanp.am.health.ge.com:1521/prd1mes
  loc_description = scdhp connection
}

### load Json

In [4]:
import Data.Aeson
import qualified Data.ByteString.Lazy as B
import Control.Applicative

input <- B.readFile "data2.json"
let mm = decode input :: Maybe [Location]
(length.unjust) mm

2596

In [5]:
l = unjust mm
let l_example =  (l!!2)

pretty l_example

In [6]:
pretty (loc2Location l_example "AAA" ["aa","bb","cc"] )

Location {
  database         = ushvr12
  dbdesc           = dev
  host             = odp-us-innovation-postgres-db.cxzrm3enxxmy.us-east-1.rds.amazonaws.com
  loc_class        = oracleAAA
   loc_name         = sedhp
   loc_directory    = /u01/app/oracle/product/19.3.0.0/dbee_1
   loc_remote_node  = ora-mke2-scanp.am.health.ge.com
   loc_remote_login = gehc_hvr5
   loc_remote_port  = 4344
   loc_db_user      = aa
   loc_description  = scdhp connection
   db_node_name     = bb
   db_instance      = cc
}